In [98]:
# pip install savReaderWriter 

import pandas as pd
import numpy as np
import savReaderWriter
import datetime as dt

with savReaderWriter.SavReader('./data/Koweps_hpc10_2015_beta1.sav', ioUtf8 = True) as reader:
    df = pd.DataFrame(reader.all(), columns = [s for s in reader.header])

display(df.head())

,h10_id,h10_ind,h10_sn,h10_merkey,h_new,h10_cobf,h10_reg5,h10_reg7,h10_din,h10_cin,...,c1007_4aq19,c1007_4aq20,c1007_4aq21,c1007_4aq22,c1007_4aq23,h10_pers_income1,h10_pers_income2,h10_pers_income3,h10_pers_income4,h10_pers_income5
0,1.0,1.0,1.0,10101.0,0.0,NaN,1.0,1.0,864.0,864.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,2.0,1.0,1.0,20101.0,0.0,NaN,1.0,1.0,600.0,600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,3.0,1.0,1.0,30101.0,0.0,NaN,1.0,1.0,1571.0,1619.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1440.0,NaN,0.0,NaN
3,4.0,1.0,1.0,40101.0,0.0,NaN,1.0,1.0,3579.0,3687.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2400.0,NaN,0.0,NaN
4,4.0,1.0,1.0,40101.0,0.0,NaN,1.0,1.0,3579.0,3687.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [289]:
# 1. 성별에 따른 월급 차이
# 과거에 비해 여성의 사회 진출이 활발하지만 직장에서의
# 위상에서는 여전히 차별이 존재하고 있는것이 사실.
# 실제로 그러한지 월급의 차이를 이용하여 사실을 확인해보자

# display(df['p1002_8aq1'].groupby(df['h10_g3']).mean())
# 1.0 남자 평균
# 2.0 여자 평균

# 컬럼 추출
성별월급 = df.filter(['h10_g3', 'p1002_8aq1'])

# 컬럼 이름 변경
성별월급df = 성별월급.rename(columns={'h10_g3': '성별', 'p1002_8aq1': '월급'})

# 성별 컬럼 값 변경
성별월급df['성별'] = np.where(성별월급df['성별'] == 1, '남자', '여자') 

display(pd.DataFrame(성별월급df['월급'].groupby(성별월급df['성별']).mean()))

,월급
성별,
남자,312.293165
여자,162.199670


In [88]:
# 2. 나이와 월급의 관계
# 몇 살 때 월급을 가장 많이 받을까? 또 그때의 월급은 얼마인가?

yearSal_filter = df.filter(['h10_g4', 'p1002_8aq1'])
yearSal_filter

yearSal_df = yearSal_filter.rename(columns={'h10_g4':'태어난년도', 'p1002_8aq1': '월급'})

# 나이 변수 추가
yy = dt.datetime.now().year
yearSal_df['나이'] = yy - yearSal_df['태어난년도'] + 1

# 결측치 확인
yearSal_df
yearSal_df.isna().sum() # 월급       12030

# 결측치 처리
yearSal_df.dropna(inplace=True)
yearSal_df.isna().sum() 

# 나이 별 월급
yearSalAvg_df = yearSal_df.filter(['나이','월급']).groupby('나이').mean()
yearSalAvg_df.sort_values(by=['월급'], axis=0, ascending=False)

# 58살 때 월급이 제일 많다.

,월급
나이,
58.0,318.677748
54.0,315.033810
49.0,314.897324
55.0,314.052143
45.0,306.467203
...,...
87.0,20.000000
90.0,20.000000
92.0,20.000000


In [207]:
# 3. 연령대에 따른 월급 차이
# 30세 미만을 초년(young), 
# 30~59세 : 중년(middle), 
# 60세 이상 : 노년(old)
# 위의 범주로 연령대에 따른 월급의 차이를 알아보자

yearSal_filter = df.filter(['h10_g4', 'p1002_8aq1'])

yearSal_df = yearSal_filter.rename(columns={'h10_g4':'태어난년도', 'p1002_8aq1': '월급'})

# 나이 변수 추가
yy = dt.datetime.now().year
yearSal_df['나이'] = yy - yearSal_df['태어난년도'] + 1

# 결측치 확인
yearSal_df.isna().sum() # 월급       12030

# 결측치 처리
yearSal_df.dropna(inplace=True)
yearSal_df.isna().sum() 

## 연령대 구분 나이 구간 조건 리스트
conditions = [ (yearSal_df['나이'] < 30), 
               (yearSal_df['나이'] < 60),
               (yearSal_df['나이'] >= 60)]

# 조건에 따라 부여될 값
level = ['초년', '중년', '노년']

# 조건에 따라 연령층 추가하기
yearSal_df['연령대'] = np.select(conditions, level)
# display(yearSal)

# 연령대별로 평균 월급 계산
yearSalAvg_df = yearSal_df.filter(['연령대','월급']).groupby('연령대').mean()
print('======연령대별로 평균 월급 계산============')
display(yearSalAvg_df)

======연령대별로 평균 월급 계산============


,월급
연령대,
노년,173.803452
중년,271.922413
초년,128.553796


In [187]:
# 4. 연령대 및 성별 월급 차이
# 성별 월급 차이는 연령대에 따라 다른 양상을 보일 수 있습니다.
# 성별 월급 차이가 연령대에 따라 다른지 분석해보자

# 기존에는 3그룹(초년,중년,노년)이었지만 이젠 6그룹으로
# 그룹핑을 해야 한다.(초년남성,초년여성,..)

성별년도급여 = df.filter(['h10_g3', 'h10_g4', 'p1002_8aq1'])
성별년도급여df = 성별년도급여.rename(columns={'h10_g3': '성별', 'h10_g4': '태어난년도', 'p1002_8aq1': '월급'})

# 성별 컬럼 값 변경
성별년도급여df['성별'] = np.where(성별년도급여df['성별'] == 1, '남자', '여자') 

# 나이 변수 추가
yy = dt.datetime.now().year
성별년도급여df['나이'] = yy - 성별년도급여df['태어난년도'] + 1
# display(성별년도급여df)

# 결측치 확인
성별년도급여df.isna().sum() # 월급       12030

# 결측치 처리
성별년도급여df.dropna(inplace=True)
성별년도급여df.isna().sum() # 월급       0

## 연령대 구분 나이 구간 조건 리스트
conditions = [ (성별년도급여df['나이'] < 30) & (성별년도급여df['성별'] == '남자'), 
               (성별년도급여df['나이'] < 30) & (성별년도급여df['성별'] == '여자'),
               (성별년도급여df['나이'] < 60) & (성별년도급여df['성별'] == '남자'), 
               (성별년도급여df['나이'] < 60) & (성별년도급여df['성별'] == '여자'),
               (성별년도급여df['나이'] >= 60) & (성별년도급여df['성별'] == '남자'), 
               (성별년도급여df['나이'] >= 60) & (성별년도급여df['성별'] == '여자')]

# 조건에 따라 부여될 값
level = ['초년남성', '초년여성', '중년남성', '중년여성', '노년남성', '노년여성']

# 조건에 따라 연령층 추가하기
성별년도급여df['연령대'] = np.select(conditions, level)
# display(성별년도급여df)

# 연령대별로 평균 월급 계산
성별년도급여Avg_df = 성별년도급여df.filter(['연령대','월급']).groupby('연령대').mean()
print('======성별&연령대별로 평균 월급 계산============')
display(성별년도급여Avg_df)

======성별&연령대별로 평균 월급 계산============


,월급
연령대,
노년남성,244.379383
노년여성,103.112764
중년남성,339.640986
중년여성,189.185980
초년남성,130.248250
초년여성,127.855052


In [215]:
!pip install xlrd

In [254]:
# 5. 직업별 월급 차이
# 어떤 직업이 월급을 가장 많이 받을까?
# 직업별 월급을 분석해 보자
# 직업코드는 제공된 Koweps_Codebook.xlsx을 이용하면 
# 편하게 코드값을 이용 할 수 있습니다.
# !pip install xlrd

# 컬럼 추출
직업월급 = df.filter(['h10_eco9', 'p1002_8aq1'])

# 컬럼 이름 변경
직업월급df = 직업월급.rename(columns={'h10_eco9': 'code_job', 'p1002_8aq1': '월급'})

# 결측치 처리
직업월급df.dropna(inplace=True)
직업월급df.isna().sum() 

# 직업 코드 읽어와서 DataFrame으로 만들기
직업df = pd.read_excel('./data/Koweps_Codebook.xlsx', sheet_name='직종 코드')
# display(직업df)
# display(직업월급df)

# 직업 코드에 해당하는 직업 입력
직업월급df = pd.merge(직업월급df, 직업df, on='code_job', how='inner')
직업월급df = 직업월급df.rename(columns={'job': '직업'})
# code_job 삭제
직업월급df.drop('code_job', axis=1, inplace=True)

직업월급avg_df = 직업월급df.groupby(['직업']).mean()

# 월급 많은 순으로 정렬
직업월급avg_df = 직업월급avg_df.sort_values(by=['월급'], axis=0, ascending=False)
display(직업월급avg_df)

,월급
직업,
금속 재료 공학 기술자 및 시험원,845.066667
의료진료 전문가,843.642857
의회의원 고위공무원 및 공공단체임원,750.000000
보험 및 금융 관리자,726.180000
제관원 및 판금원,572.406667
...,...
약사 및 한약사,89.000000
청소원 및 환경 미화원,88.787747
기타 서비스관련 단순 종사원,88.221010


In [283]:
# 6. 성별 직업 빈도
# 성별로 어떤 직업이 가장 많을까?

# 컬럼 추출
성별직업 = df.filter(['h10_g3', 'h10_eco9'])

# 컬럼 이름 변경
성별직업df = 성별직업.rename(columns={'h10_g3': '성별', 'h10_eco9': 'code_job'})

# 성별 컬럼 값 변경
성별직업df['성별'] = np.where(성별직업df['성별'] == 1, '남자', '여자') 

# 결측치 처리
성별직업df.dropna(inplace=True)
성별직업df.isna().sum() 

# display(성별직업df)

# 직업 코드 읽어와서 DataFrame으로 만들기
직업df = pd.read_excel('./data/Koweps_Codebook.xlsx', sheet_name='직종 코드')

# 직업 코드에 해당하는 직업 입력
성별직업df= pd.merge(성별직업df, 직업df, on='code_job', how='inner')
성별직업df = 성별직업df.rename(columns={'job': '직업'})

# code_job 삭제
성별직업df.drop('code_job', axis=1, inplace=True)

# 성별로 직업 세기
성별직업count = pd.DataFrame(성별직업df.groupby(['성별','직업']).size().reset_index(name='인원'))

# 인원이 많은 직업부터 정렬
성별직업count = 성별직업count.sort_values(by=['인원'], axis=0, ascending=False)
display(성별직업count)

,성별,직업,인원
217,여자,작물재배 종사자,680
100,남자,작물재배 종사자,640
97,남자,자동차 운전원,251
231,여자,청소원 및 환경 미화원,228
172,여자,매장 판매 종사자,221
...,...,...,...
222,여자,전기공,1
169,여자,도장 및 도금기 조작원,1
173,여자,목재 및 종이 관련 기계조작원,1
223,여자,정보 시스템 운영자,1


In [328]:
# 7. 종교 유무에 따른 이혼율
# 종교가 있는 사람들이 이혼을 덜 할까??

# 컬럼 추출
종교이혼 = df.filter(['h10_g11', 'h10_g10'])

# 칼럼 이름 변경
종교이혼df = 종교이혼.rename(columns={'h10_g11':'종교', 'h10_g10':'혼인 상태'})

# 종교 컬럼 값 변경
종교이혼df['종교'] = np.where(종교이혼df['종교'] == 1, '유교', '무교') 

## 혼인 상태 구분 조건 리스트
conditions = [ (종교이혼df['혼인 상태'] == 0), 
               (종교이혼df['혼인 상태'] == 1), 
               (종교이혼df['혼인 상태'] == 2),
               (종교이혼df['혼인 상태'] == 3), 
               (종교이혼df['혼인 상태'] == 4), 
               (종교이혼df['혼인 상태'] == 5),
               (종교이혼df['혼인 상태'] == 6)]

# 조건에 따라 부여될 값
level = ['비해당(18세 미만)', '유배우', '사별', '이혼', '별거', '미혼', '기타']

종교이혼df['혼인 상태'] = np.select(conditions, level)
display(종교이혼df)


###
# 종교가 있고 이혼한 사람
유교이혼_count = sum((종교이혼df['종교'] == '유교') & (종교이혼df['혼인 상태'] == '이혼'))
print(유교이혼_count) # 16664

# 종교가 있고 결혼한 사람 (전체 - 비해당 - 미혼 - 기타)
유교결혼_count = sum((종교이혼df['종교'] == '유교')\
               & (종교이혼df['혼인 상태'] != '비해당(18세 미만)')\
               & (종교이혼df['혼인 상태'] != '미혼')\
               & (종교이혼df['혼인 상태'] != '기타)'))
print(유교결혼_count) # 328

# 종교가 없고 이혼한 사람
무교이혼_count = sum((종교이혼df['종교'] == '무교') & (종교이혼df['혼인 상태'] == '이혼'))
print(무교이혼_count) # 5881

# 종교가 없고 결혼한 사람 (전체 - 비해당 - 미혼 - 기타)
무교결혼_count = sum((종교이혼df['종교'] == '무교')\
               & (종교이혼df['혼인 상태'] != '비해당(18세 미만)')\
               & (종교이혼df['혼인 상태'] != '미혼')\
               & (종교이혼df['혼인 상태'] != '기타)'))
print(무교결혼_count) # 5489

print('유교인 사람의 이혼률 : {}'.format(  (유교이혼_count/유교결혼_count)*100)  )
print('무교인 사람의 이혼률 : {}'.format(  (무교이혼_count/무교결혼_count)*100)  )

,종교,혼인 상태
0,무교,사별
1,무교,사별
2,무교,사별
3,유교,이혼
4,유교,사별
...,...,...
16659,유교,유배우
16660,유교,미혼
16661,유교,미혼
16662,유교,비해당(18세 미만)


328
5881
384
5489
유교인 사람의 이혼률 : 5.577282775038259
무교인 사람의 이혼률 : 6.995809801421024


In [206]:
# 8. 지역별 연령대 비율
# 노년층이 많은 지역은 어디일까?


# 컬럼 추출
지역년도 = df.filter(['h10_g4', 'h10_reg7'])

# 컬럼 이름 변경
지역년도df = 지역년도.rename(columns={'h10_g4': '태어난년도', 'h10_reg7': '지역코드'})

# 나이 컬럼 추가
지역년도df['나이'] = dt.datetime.now().year - 지역년도df['태어난년도'] + 1

## 연령대 구분 나이 구간 조건 리스트
conditions = [ (지역년도df['나이'] < 30), 
               (지역년도df['나이'] < 60),
               (지역년도df['나이'] >= 60)]

# 조건에 따라 부여될 값
level = ['초년', '중년', '노년']

# 조건에 따라 연령층 추가하기
지역년도df['연령대'] = np.select(conditions, level)
# display(yearSal)

# 태어난 년도, 나이 컬럼 삭제
지역년도df.drop('태어난년도', axis=1, inplace=True)

# 결과 확인
# 지역년도df


# 지역 코드와 매칭될 코드 표
지역코드표df = pd.DataFrame({'지역코드': list(range(1,8)), 
        '지역': ["서울", "인천,경기", "부산,경남,울산", "대구,경북", "대전,충남", "강원,충북", "광주,전남,전북,제주"]})
지역코드표df

# 원래의 데이터 프레임에 지역 코드표 병합
지역나이df = pd.merge(지역년도df, 지역코드표df, how='outer')

# 지역, 연령대 별로 인원 세기
지역별연령대df = 지역나이df.filter(['지역','연령대','나이']).groupby(['지역','연령대'], as_index=False).count()
지역별연령대df.rename(columns={'나이': '인원'}, inplace=True) 
지역별연령대df

# 노년층만 추출 후 인원이 많은 순으로 정렬
지역별노년인원df = 지역별연령대df.loc[지역별연령대df['연령대'] == '노년']
지역별노년인원df = 지역별노년인원df.sort_values(by=['인원'], axis=0, ascending=False)

display(지역별노년인원df)

,지역,연령대,인원
3,"광주,전남,전북,제주",노년,1389
18,"인천,경기",노년,1333
12,"부산,경남,울산",노년,1307
6,"대구,경북",노년,1051
15,서울,노년,999
9,"대전,충남",노년,633
0,"강원,충북",노년,630
